# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de postprocesmiento de la grilla espacial

En este script se tomará la grilla espacial generada en el notebook *main* y se buscarán las celdas con mejor potencial ornitológico dentro de cada cluster.

## Importaciones

In [ ]:
import pandas as pd
import geopandas as gpd
import sys
from pathlib import Path
from pyogrio.errors import DataSourceError
import folium
from folium.plugins import Fullscreen

# Modulos propios
sys.path.append(str(Path('..')/'src'))

import utils

## Carga de datos

In [17]:
ruta_grilla = '../data/processed/grilla_tdf_clusters.gpkg'
ruta_registros = '../data/interim/grilla_tdf_spp.csv'

try:
    grilla = gpd.read_file(ruta_grilla)
except DataSourceError:
    print('No se encuentra la grilla, revise la ruta')

try:
    registros = pd.read_csv(ruta_registros)
except FileNotFoundError:
    print('No se encuentran los registros, revise la ruta')

## Cálculo de nuevas variables

#### Crear un nuevo dataframe para almacenar variables temporales
Aquí se almacenarán variables para permitir calcular las variables de interés antes de incorporarlas a la grilla. La inicializo con el número de celda y de cluster

In [18]:
df_calc = grilla[['grid_id', 'GaussianMixture']].copy()
# Filtrar las filas donde haya datos del cluster
df_calc = df_calc[df_calc['GaussianMixture'].notna()]
# Renombrar la columna ‘GaussianMixture’ a ‘cluster’
df_calc = df_calc.rename(columns={'GaussianMixture': 'cluster'})

### Celdas de cada cluster con mayor riqueza de aves registradas
Calculo la cantidad de especies registradas (riqueza) en cada celda y la proporción de registros. Luego asigno un puntaje a cada celda según la cantidad de especies y la proporción de registros en esa celda, siendo independientes los puntajes para cada cluster. Las especies de la lista de prioritarias otorgan un puntaje doble.

#### Copiar el dataframe y duplicar el valor de las especies de interés
Al duplicar la cantidad de registros de las especies de interés se les asigna mayor peso

Los observadores de aves en muchos casos viajan a determinados lugares buscando ciertas especies en particular, además de tratar de registrar toda la avifauna posible. Para Tierra del Fuego, elaboré una lista de las especies más buscadas y las especies endémicas o que dificilmente se observan en otros sitios, según una encuesta realizada al Club de Observadores de Aves de Ushuaia, chat GPT y conocimiento propio Estas especies fueron: 
- Carpintero gigante (*Campephilus magellanicus*)
- Cóndor andino (*Vultur gryphus*)
- Caranca (*Chloephaga hybrida*)
- Cauquén colorado (*Chloephaga rubidiceps*)
- Matamico blanco (*Daptrius albogularis*)
- Carancho austral (*Daptrius australis*)
- Remolinera negra (*Cinclodes antarcticus*)
- Quetro austral (*Tachyeres pteneres*)
- Caminera patagónica (*Geositta antarctica*)
- Playero Rojizo (*Calidris canutus*)
- Pingüino Rey (*Aptenodytes patagonicus*)
- Albatros de Ceja Negra (*Thalassarche melanophris*)

Al duplicar la cantidad de registros de las especies de interés se les asigna mayor peso.

In [19]:
lista_spp_buscadas = [
    "Campephilus magellanicus",
    "Vultur gryphus",
    "Chloephaga hybrida",
    "Chloephaga rubidiceps",
    "Daptrius albogularis",
    "Daptrius australis",
    "Cinclodes antarcticus",
    "Tachyeres pteneres",
    "Geositta antarctica",
    "Calidris canutus",
    "Aptenodytes patagonicus",
    "Thalassarche melanophris"
]

# Convertir la columna 'grid_id' de registros en índice
reg_idx = registros.set_index('grid_id')

# Revisar que las especies buscadas estén en los registros bien escritas
for spp in lista_spp_buscadas:
    print(spp in reg_idx.columns)

True
True
True
True
True
True
True
True
True
True
True
True


In [20]:
# Calcular, para cada especie (columna), el máximo número de registros en cualquier celda
max_por_especie = reg_idx.max(axis=0)
# Construir un DataFrame de proporciones fila‒columna:
df_prop = reg_idx.div(max_por_especie, axis=1)

# Crear una Serie de pesos por especie: 1 por defecto, 2 si está en la lista de spp buscadas
pesos = pd.Series(1.0, index=reg_idx.columns)
pesos.loc[lista_spp_buscadas] = 2.0
# Multiplicar cada columna de proporciones por su peso
df_ponderado = df_prop * pesos

# Para cada fila (celda), sumar las proporciones ponderadas → riqueza ponderada
reg_idx = reg_idx.assign(riqueza_ponderada = df_ponderado.sum(axis=1))

##### Agregar la riqueza ponderada al df temporal

In [21]:
df_calc['riqueza_ponderada'] = df_calc['grid_id'].map(reg_idx['riqueza_ponderada'])

##### Calcular el puntaje de riqueza por celda de cada cluster
El puntaje va entre 0 (sin registros) y 1 (la celda con mayor riqueza ponderada del cluster)

In [22]:
# Calcular, para cada fila, el máximo de riqueza ponderada en su cluster
max_por_cluster = df_calc.groupby('cluster')['riqueza_ponderada'] \
                         .transform('max')
# Dividir la riqueza de cada fila por ese máximo
df_calc['score_riqueza'] = df_calc['riqueza_ponderada'] / max_por_cluster

In [23]:
df_calc

,grid_id,cluster,riqueza_ponderada,score_riqueza
0,25,2.0,0.839969,0.009046
1,26,2.0,0.255404,0.002750
2,27,2.0,0.743776,0.008010
3,28,2.0,0.019943,0.000215
5,70,2.0,1.042126,0.011223
...,...,...,...,...
909,2300,1.0,0.055057,0.001281
910,2301,1.0,0.348419,0.008110
912,2346,2.0,0.034483,0.000371
914,2392,1.0,4.751054,0.110583


## Agregar las nuevas variables a la grilla espacial

In [24]:
# Agregar el resultado del modelo a la grilla espacial
gdf_grid = utils.grillar_res_mod(
    grilla=grilla,
    id_grid=df_calc['grid_id'],
    clusters=df_calc['score_riqueza'],
    nombre='score_riqueza'
)

In [25]:
# Guardo el resultado en un nuevo .gpkg
ruta_salida = "../data/processed/grilla_riqueza.gpkg"
gdf_grid.to_file(ruta_salida, driver="GPKG", layer="grilla_clusters")

print("Grilla con clusters guardada en:", ruta_salida)

Grilla con clusters guardada en: ../data/processed/grilla_riqueza.gpkg


## Visualizar el mapa resultante
La visualización del mapa final se realizó con Qgis y si se puede observar más abajo. Aquí se muestra una visualización más sencilla del resultado.

#### Mapa interactivo

In [97]:
# Convertir a EPSG:4326 (necesario para folium)
gdf = gdf_grid.to_crs(epsg=4326)

# Definir colores según los clusters (las claves son strings)
colores = {
    '0.0': '#49E973',
    '1.0': '#A8B47E',
    '2.0': '#51D3E1',
    'Sin datos': '#000000'
}

# Asegurarte que GaussianMixture sea string y no categoría
gdf['GaussianMixture'] = gdf['GaussianMixture'].fillna('Sin datos').astype(str)
gdf['score_riqueza'].fillna(0.0)

# Centrar mapa
centro = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]

# Crear mapa
m = folium.Map(location=centro, zoom_start=7, tiles="cartodbpositron")
Fullscreen().add_to(m)

# Función de estilo para folium: las claves deben estar en camelCase
def estilo(feature):
    cluster = feature['properties']['GaussianMixture']
    color = colores.get(cluster, '#f0f0f0')
    return {
        'fillColor': color,          
        'color': color,            # Color del borde
        'weight': 0.5,
        'fillOpacity': 0.7
    }

# Añadir capa GeoJson
folium.GeoJson(
    gdf,
    style_function=estilo,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['GaussianMixture', 'score_riqueza'],
        aliases=['Cluster:', 'Puntaje de riqueza:'],
        localize=True
    ),
    name='Grilla'
).add_to(m)

folium.TileLayer(
    tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(m)


# Guardar
m.save("../reports/figures/mapa_interactivo.html")

# Ver el mapa
m



C:\Users\pablo\AppData\Local\Temp\ipykernel_10180\3451400746.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centro = [gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean()]
